In [ ]:
import json
from result_utils import create_dict_exps, create_dict_results_total, create_dict_results_entropy
import numpy as np
import sumolib
from numpyencoder import NumpyEncoder

## Compute Results

In [ ]:
navigators = ["w1", "w2p5"]#, "w5", "w7p5", "w10", "w12p5", "w15", "w17p5", "w20"]

city = "milan"

folder_experiments = "../sim_outputs"

output_folder = "../results/"

print(city)
print(len(set(navigators)))

In [ ]:
# road network path

road_network_path = f"../data/{city}/{city}_road_network.net.xml"
road_network_int = sumolib.net.readNet(road_network_path, withInternal=True)

len(road_network_int.getEdges())

In [ ]:
dict_length_edges = {}

for e in road_network_int.getEdges():
    dict_length_edges[e.getID()] = e.getLength()

In [ ]:
%%time

final_dict = {}

measures_edge = ["total_co2"]

measures_vehicle = ["total_traveltime"]

for nav in navigators:
    
    print(f"Navigator: {nav}")
        
    final_dict[nav] = {"total":{}, "entropy":{}, "entropy_normalized":{}}

    
    folder_experiments_nav = f"{folder_experiments}/{city}/{nav}/"
    
    dict_exps_nav = create_dict_exps(folder_experiments_nav, "vehicle")
    
    
    # edge-based
    print("edge-based measures")
    for m in measures_edge:       
        
        for what in final_dict[nav].keys():
            
            operation = what.split("_")[0]
            fun_to_apply = eval(f"create_dict_results_{operation}")
            
            if "normalized" in what:
                
                print(f"Computing {what} {m} normalized")

                final_dict[nav][what][m] = fun_to_apply(dict_exps_nav, folder_experiments_nav, 
                                                        m, "edge", normalize_by=dict_length_edges)
            else:
                print(f"Computing {what} {m}")

                final_dict[nav][what][m] = fun_to_apply(dict_exps_nav, folder_experiments_nav, 
                                                        m, "edge")
    
    # vehicle-based
    print("vehicle-based measures")
    for m in measures_vehicle:  
                
        for what in final_dict[nav].keys():
        
            operation = what.split("_")[0]
            fun_to_apply = eval(f"create_dict_results_{operation}")

            if "normalized" in what:
                pass
            else:
                print(f"Computing {what} {m}")
                final_dict[nav][what][m] = fun_to_apply(dict_exps_nav, folder_experiments_nav, 
                                                        m, "vehicle")
    
    
    '''
    folder_nav_baseline = f"{folder_experiments}/{city}/baselines/"
    baseline = [f for f in os.listdir(folder_nav_baseline) if nav+"_" in f]
    d = {"baseline":{0:baseline[0]}}
    
    print(d)
    
    # baselines
    print("Baselines")
    
    for m in measures_edge:
                
        for what in final_dict[nav].keys():
                        
            operation = what.split("_")[0]
            fun_to_apply = eval(f"create_dict_results_{operation}")
            
            if "normalized" in what:
                print(f"Computing {what} {m} normalized")

                final_dict[nav][what][m]["baseline"] = fun_to_apply(d, folder_nav_baseline, 
                                                        m, "edge", normalize_by=dict_length_edges)
            else:
                print(f"Computing {what} {m}")
                final_dict[nav][what][m]["baseline"] = fun_to_apply(d, folder_nav_baseline, 
                                                        m, "edge")
        
            
        
    for m in measures_vehicle:  
                             
        for what in final_dict[nav].keys():
        
            operation = what.split("_")[0]
            fun_to_apply = eval(f"create_dict_results_{operation}")

            if "normalized" in what:
                pass
            else:
                final_dict[nav][what][m]["baseline"] = fun_to_apply(d, folder_nav_baseline, 
                                                        m, "vehicle")
    '''

In [ ]:
# save the json

with open(f'{output_folder}results_{city}.json', 'w') as outfile:
    json.dump(final_dict, outfile, cls=NumpyEncoder)